In [28]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow import keras
import os
import json
import time
from datetime import datetime
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as peak_signal_noise_ratio
import lpips

In [29]:
class SuperResolutionGUI:
    def _init_(self, root):
        self.root = root
        self.root.title("Super Resolution - ESPCN")
        self.root.geometry("1200x800")
        
        # Variables
        self.model = None
        self.lpips_model = None
        self.input_image = None
        self.output_image = None
        self.scale_factor = 4
        
        # UI Variables
        self.input_image_tk = None
        self.output_image_tk = None
        
        # Initialize
        self.setup_ui()
        self.load_model()
        
    def setup_ui(self):
        """Setup GUI layout"""
        # Main frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Configure grid weights
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(2, weight=1)
        
        # Title
        title_label = ttk.Label(main_frame, text="Super Resolution dengan ESPCN", 
                               font=("Arial", 16, "bold"))
        title_label.grid(row=0, column=0, columnspan=3, pady=(0, 20))
        
        # Control Panel
        control_frame = ttk.LabelFrame(main_frame, text="Controls", padding="10")
        control_frame.grid(row=1, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        # Upload button
        ttk.Button(control_frame, text="Upload Image", 
                  command=self.upload_image).grid(row=0, column=0, padx=(0, 10))
        
        # Process button
        self.process_btn = ttk.Button(control_frame, text="Process Image", 
                                     command=self.process_image, state="disabled")
        self.process_btn.grid(row=0, column=1, padx=(0, 10))
        
        # Save button
        self.save_btn = ttk.Button(control_frame, text="Save Result", 
                                  command=self.save_result, state="disabled")
        self.save_btn.grid(row=0, column=2, padx=(0, 10))
        
        # Progress bar
        self.progress = ttk.Progressbar(control_frame, mode='indeterminate')
        self.progress.grid(row=0, column=3, sticky=(tk.W, tk.E), padx=(10, 0))
        control_frame.columnconfigure(3, weight=1)
        
        # Image Display Frame
        image_frame = ttk.Frame(main_frame)
        image_frame.grid(row=2, column=0, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        image_frame.columnconfigure(0, weight=1)
        image_frame.columnconfigure(1, weight=1)
        image_frame.rowconfigure(0, weight=1)
        
        # Input Image Frame
        input_frame = ttk.LabelFrame(image_frame, text="Input Image", padding="10")
        input_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(0, 5))
        
        self.input_label = ttk.Label(input_frame, text="No image selected", 
                                    background="white", anchor="center")
        self.input_label.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        input_frame.columnconfigure(0, weight=1)
        input_frame.rowconfigure(0, weight=1)
        
        # Output Image Frame
        output_frame = ttk.LabelFrame(image_frame, text="Output Image", padding="10")
        output_frame.grid(row=0, column=1, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(5, 0))
        
        self.output_label = ttk.Label(output_frame, text="No output yet", 
                                     background="white", anchor="center")
        self.output_label.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        output_frame.columnconfigure(0, weight=1)
        output_frame.rowconfigure(0, weight=1)
        
        # Metrics Frame
        metrics_frame = ttk.LabelFrame(main_frame, text="Metrics & Information", padding="10")
        metrics_frame.grid(row=3, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(10, 0))
        
        # Metrics display
        self.metrics_text = tk.Text(metrics_frame, height=8, width=80)
        self.metrics_text.grid(row=0, column=0, sticky=(tk.W, tk.E))
        
        # Scrollbar for metrics
        scrollbar = ttk.Scrollbar(metrics_frame, orient="vertical", command=self.metrics_text.yview)
        scrollbar.grid(row=0, column=1, sticky=(tk.N, tk.S))
        self.metrics_text.configure(yscrollcommand=scrollbar.set)
        
        metrics_frame.columnconfigure(0, weight=1)
        
        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_bar = ttk.Label(main_frame, textvariable=self.status_var, 
                              relief=tk.SUNKEN, anchor=tk.W)
        status_bar.grid(row=4, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(10, 0))
        
    def load_model(self):
        """Load trained ESPCN model"""
        try:
            model_path = "models/espcn_final.h5"
            if os.path.exists(model_path):
                self.model = keras.models.load_model(model_path)
                self.status_var.set("Model loaded successfully")
                self.log_message("✅ ESPCN model loaded successfully")
                
                # Load LPIPS model
                self.lpips_model = lpips.LPIPS(net='alex')
                self.log_message("✅ LPIPS model loaded successfully")
                
            else:
                self.status_var.set("Model not found")
                self.log_message("❌ Model file not found. Please run training first.")
                messagebox.showerror("Error", "Model file not found. Please run training notebook first.")
                
        except Exception as e:
            self.status_var.set("Error loading model")
            self.log_message(f"❌ Error loading model: {str(e)}")
            messagebox.showerror("Error", f"Error loading model: {str(e)}")
    
    def log_message(self, message):
        """Add message to metrics display"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.metrics_text.insert(tk.END, f"[{timestamp}] {message}\n")
        self.metrics_text.see(tk.END)
        self.root.update()
    
    def upload_image(self):
        """Upload and display input image"""
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        
        if file_path:
            try:
                # Load image
                image = cv2.imread(file_path)
                if image is None:
                    raise ValueError("Cannot read image file")
                
                # Convert BGR to RGB
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                self.input_image = image
                
                # Display image
                self.display_image(image, self.input_label, "input")
                
                # Enable process button
                self.process_btn.config(state="normal")
                
                # Log info
                h, w = image.shape[:2]
                self.log_message(f"📁 Image loaded: {os.path.basename(file_path)}")
                self.log_message(f"📏 Image size: {w}x{h} pixels")
                
                self.status_var.set("Image loaded successfully")
                
            except Exception as e:
                self.log_message(f"❌ Error loading image: {str(e)}")
                messagebox.showerror("Error", f"Error loading image: {str(e)}")
    
    def display_image(self, image, label, image_type):
        """Display image in label with proper scaling"""
        # Convert to PIL Image
        pil_image = Image.fromarray(image.astype(np.uint8))
        
        # Calculate display size (max 400x400)
        display_size = 400
        w, h = pil_image.size
        
        if w > h:
            new_w = display_size
            new_h = int(h * display_size / w)
        else:
            new_h = display_size
            new_w = int(w * display_size / h)
        
        # Resize for display
        pil_image = pil_image.resize((new_w, new_h), Image.Resampling.LANCZOS)
        
        # Convert to PhotoImage
        photo = ImageTk.PhotoImage(pil_image)
        
        # Update label
        label.configure(image=photo, text="")
        label.image = photo  # Keep a reference
        
        # Store reference
        if image_type == "input":
            self.input_image_tk = photo
        else:
            self.output_image_tk = photo
    
    def process_image(self):
        """Process image with ESPCN model"""
        if self.input_image is None or self.model is None:
            messagebox.showerror("Error", "Please load an image and model first")
            return
        
        try:
            # Start progress
            self.progress.start()
            self.status_var.set("Processing image...")
            self.log_message("🔄 Processing image with ESPCN...")
            
            # Prepare input
            input_image = self.input_image.copy()
            
            # Resize to multiple of scale factor if needed
            h, w = input_image.shape[:2]
            new_h = (h // self.scale_factor) * self.scale_factor
            new_w = (w // self.scale_factor) * self.scale_factor
            
            if new_h != h or new_w != w:
                input_image = cv2.resize(input_image, (new_w, new_h))
                self.log_message(f"📏 Resized to {new_w}x{new_h} for processing")
            
            # Create low-res version
            lr_h, lr_w = new_h // self.scale_factor, new_w // self.scale_factor
            lr_image = cv2.resize(input_image, (lr_w, lr_h), interpolation=cv2.INTER_CUBIC)
            
            # Normalize
            lr_normalized = lr_image.astype(np.float32) / 255.0
            
            # Add batch dimension
            lr_batch = np.expand_dims(lr_normalized, axis=0)
            
            # Predict
            start_time = time.time()
            sr_batch = self.model.predict(lr_batch, verbose=0)
            inference_time = time.time() - start_time
            
            # Post-process
            sr_image = sr_batch[0]
            sr_image = np.clip(sr_image, 0, 1)
            sr_image = (sr_image * 255).astype(np.uint8)
            
            self.output_image = sr_image
            
            # Display result
            self.display_image(sr_image, self.output_label, "output")
            
            # Calculate metrics (compare with bicubic upscaling)
            bicubic_upscaled = cv2.resize(lr_image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
            
            # Calculate metrics
            psnr_val = self.calculate_psnr(input_image, sr_image)
            ssim_val = self.calculate_ssim(input_image, sr_image)
            
            psnr_bicubic = self.calculate_psnr(input_image, bicubic_upscaled)
            ssim_bicubic = self.calculate_ssim(input_image, bicubic_upscaled)
            
            # Log results
            self.log_message("✅ Processing completed!")
            self.log_message(f"⏱  Inference time: {inference_time:.4f} seconds")
            self.log_message(f"📊 ESPCN vs Original:")
            self.log_message(f"   PSNR: {psnr_val:.4f} dB")
            self.log_message(f"   SSIM: {ssim_val:.4f}")
            self.log_message(f"📊 Bicubic vs Original:")
            self.log_message(f"   PSNR: {psnr_bicubic:.4f} dB")
            self.log_message(f"   SSIM: {ssim_bicubic:.4f}")
            
            improvement_psnr = psnr_val - psnr_bicubic
            improvement_ssim = ssim_val - ssim_bicubic
            
            self.log_message(f"📈 Improvement over Bicubic:")
            self.log_message(f"   PSNR: {improvement_psnr:+.4f} dB")
            self.log_message(f"   SSIM: {improvement_ssim:+.4f}")
            
            # Enable save button
            self.save_btn.config(state="normal")
            
            self.status_var.set("Processing completed successfully")
            
        except Exception as e:
            self.log_message(f"❌ Error during processing: {str(e)}")
            messagebox.showerror("Error", f"Error during processing: {str(e)}")
            self.status_var.set("Processing failed")
        
        finally:
            self.progress.stop()
    
    def calculate_psnr(self, img1, img2):
        """Calculate PSNR between two images"""
        try:
            return psnr(img1, img2, data_range=255)
        except:
            return 0
    
    def calculate_ssim(self, img1, img2):
        """Calculate SSIM between two images"""
        try:
            return ssim(img1, img2, data_range=255, channel_axis=2)
        except:
            return 0
    
    def save_result(self):
        """Save the output image"""
        if self.output_image is None:
            messagebox.showerror("Error", "No output image to save")
            return
        
        file_path = filedialog.asksaveasfilename(
            title="Save Result",
            defaultextension=".jpg",
            filetypes=[("JPEG files", ".jpg"), ("PNG files", ".png")]
        )
        
        if file_path:
            try:
                # Convert RGB to BGR for OpenCV
                output_bgr = cv2.cvtColor(self.output_image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(file_path, output_bgr)
                
                self.log_message(f"💾 Result saved: {os.path.basename(file_path)}")
                self.status_var.set("Result saved successfully")
                
            except Exception as e:
                self.log_message(f"❌ Error saving result: {str(e)}")
                messagebox.showerror("Error", f"Error saving result: {str(e)}")

In [30]:
def create_comparison_window():
    """Create a comparison window for different methods"""
    comparison_window = tk.Toplevel()
    comparison_window.title("Method Comparison")
    comparison_window.geometry("1000x600")
    
    # This would show side-by-side comparison of different methods
    # Implementation would be similar to main window but with multiple outputs
    
    ttk.Label(comparison_window, text="Comparison Window", 
              font=("Arial", 14, "bold")).pack(pady=20)
    
    ttk.Label(comparison_window, text="This feature would show comparison between:\n"
                                    "• Bilinear Upscaling\n"
                                    "• Bicubic Upscaling\n"
                                    "• ESPCN\n"
                                    "• Ground Truth (if available)").pack(pady=10)

In [31]:
def main():
    """Main function to run the GUI"""
    print("🚀 Starting Super Resolution GUI...")
    
    # Create root window
    root = tk.Tk()
    
    # Set window icon (optional)
    try:
        root.iconbitmap("icon.ico")  # Add icon if available
    except:
        pass
    
    # Create and run application
    app = SuperResolutionGUI(root)
    
    # Add menu bar
    menubar = tk.Menu(root)
    root.config(menu=menubar)
    
    # File menu
    file_menu = tk.Menu(menubar, tearoff=0)
    menubar.add_cascade(label="File", menu=file_menu)
    file_menu.add_command(label="Open Image", command=app.upload_image)
    file_menu.add_command(label="Save Result", command=app.save_result)
    file_menu.add_separator()
    file_menu.add_command(label="Exit", command=root.quit)
    
    # Tools menu
    tools_menu = tk.Menu(menubar, tearoff=0)
    menubar.add_cascade(label="Tools", menu=tools_menu)
    tools_menu.add_command(label="Method Comparison", command=create_comparison_window)
    
    # Help menu
    help_menu = tk.Menu(menubar, tearoff=0)
    menubar.add_cascade(label="Help", menu=help_menu)
    help_menu.add_command(label="About", 
                         command=lambda: messagebox.showinfo("About", 
                                                            "Super Resolution GUI\n"
                                                            "Using ESPCN Model\n"
                                                            "Version 1.0"))
    
    print("✅ GUI initialized successfully")
    print("📝 Instructions:")
    print("   1. Click 'Upload Image' to select an image")
    print("   2. Click 'Process Image' to enhance resolution")
    print("   3. Click 'Save Result' to save the output")
    print("   4. Check metrics in the bottom panel")
    
    # Run the application
    root.mainloop()


In [32]:
# Check if running in Jupyter notebook
if __name__ == "_main_":
    try:
        # For Jupyter notebook, we need to run in a separate thread
        import threading
        
        def run_gui():
            main()
        
        # Start GUI in separate thread
        gui_thread = threading.Thread(target=run_gui)
        gui_thread.daemon = True
        gui_thread.start()
        
        print("GUI started in background thread")
        print("Note: In Jupyter, the GUI runs in a separate thread")
        print("You can continue using other cells while GUI is running")
        
    except Exception as e:
        print(f"Error starting GUI: {e}")
        print("Try running this in a regular Python environment")

In [33]:
# Alternative: Simple function to run GUI directly
def run_gui_direct():
    """Run GUI directly (for non-Jupyter environments)"""
    main()

# Uncomment the line below to run GUI directly
# run_gui_direct()

In [34]:
# Test function untuk cek model
def pixel_shuffle(x):
    return tf.nn.depth_to_space(x, 4)  # Ganti 4 jika pakai SCALE_FACTOR lain

def pixel_shuffle_output_shape(input_shape):
    batch, h, w, c = input_shape
    r = 4
    return (batch, h * r, w * r, c // (r * r))

def test_model_loading():
    """Test if model can be loaded"""
    try:
        model_path = "models/espcn_final.h5"
        if os.path.exists(model_path):
            model = keras.models.load_model(
                model_path,
                custom_objects={
                    'pixel_shuffle': pixel_shuffle,
                    'pixel_shuffle_output_shape': pixel_shuffle_output_shape,
                    'mse': tf.keras.losses.MeanSquaredError()
    }
)

            print("✅ Model loaded successfully")
            print(f"Model input shape: {model.input_shape}")
            print(f"Model output shape: {model.output_shape}")
            
            # Test with dummy input
            dummy_input = np.random.random((1, 64, 64, 3)).astype(np.float32)
            output = model.predict(dummy_input)
            print(f"✅ Test prediction successful, output shape: {output.shape}")
            
        else:
            print("❌ Model file not found")
            print("Please run the training notebook first")
            
    except Exception as e:
        print(f"❌ Error loading model: {e}")

# Test model loading
test_model_loading()


✅ Model loaded successfully
Model input shape: (None, 64, 64, 3)
Model output shape: (None, 256, 256, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
✅ Test prediction successful, output shape: (1, 256, 256, 3)


In [35]:
print("="*60)
print("✅ GUI APPLICATION SETUP COMPLETED")
print("="*60)
print("\nHow to use:")
print("1. Run the training notebook first to create the model")
print("2. Run main() or run_gui_direct() to start the GUI")
print("3. Upload an image and click 'Process Image'")
print("4. View results and metrics in the interface")
print("5. Save the enhanced image using 'Save Result'")
print("\nFeatures:")
print("• Upload any image format (JPG, PNG, etc.)")
print("• 4x super resolution using ESPCN")
print("• Real-time metrics (PSNR, SSIM)")
print("• Comparison with bicubic upscaling")
print("• Save results in multiple formats")
print("\n🚀 Ready to enhance your images!")


✅ GUI APPLICATION SETUP COMPLETED

How to use:
1. Run the training notebook first to create the model
2. Run main() or run_gui_direct() to start the GUI
3. Upload an image and click 'Process Image'
4. View results and metrics in the interface
5. Save the enhanced image using 'Save Result'

Features:
• Upload any image format (JPG, PNG, etc.)
• 4x super resolution using ESPCN
• Real-time metrics (PSNR, SSIM)
• Comparison with bicubic upscaling
• Save results in multiple formats

🚀 Ready to enhance your images!
